In [1]:
from theano.sandbox import cuda
cuda.use('gpu0')

WARNING (theano.sandbox.cuda): The cuda backend is deprecated and will be removed in the next release (v0.10).  Please switch to the gpuarray backend. You can get more information about how to switch at this URL:
 https://github.com/Theano/Theano/wiki/Converting-to-the-new-gpu-back-end%28gpuarray%29

Using gpu device 0: GeForce GTX 1070 (CNMeM is enabled with initial size: 95.0% of memory, cuDNN Mixed dnn version. The header is from one version, but we link with a different version (5005, 5105))
WARNING (theano.sandbox.cuda): The cuda backend is deprecated and will be removed in the next release (v0.10).  Please switch to the gpuarray backend. You can get more information about how to switch at this URL:
 https://github.com/Theano/Theano/wiki/Converting-to-the-new-gpu-back-end%28gpuarray%29



In [2]:
%matplotlib inline
import utils; reload(utils)
from utils import *
from __future__ import division, print_function

Using Theano backend.


In [3]:
model_path = '/home/xelburn/ML_Data/imdb/models/'

## Setup data

We're going to look at the IMDB dataset, which contains movie reviews from IMDB, along with their sentiment. Keras comes with some helpers for this dataset.

In [4]:
from keras.datasets import imdb
# word index: maps an integer to every single word
idx = imdb.get_word_index()

In [5]:
type(imdb)

module

In [6]:
idx

{'fawn': 34701,
 'tsukino': 52006,
 'nunnery': 52007,
 'sonja': 16816,
 'vani': 63951,
 'woods': 1408,
 'spiders': 16115,
 'hanging': 2345,
 'woody': 2289,
 'trawling': 52008,
 "hold's": 52009,
 'comically': 11307,
 'localized': 40830,
 'disobeying': 30568,
 "'royale": 52010,
 "harpo's": 40831,
 'canet': 52011,
 'aileen': 19313,
 'acurately': 52012,
 "diplomat's": 52013,
 'rickman': 25242,
 'arranged': 6746,
 'rumbustious': 52014,
 'familiarness': 52015,
 "spider'": 52016,
 'hahahah': 68804,
 "wood'": 52017,
 'transvestism': 40833,
 "hangin'": 34702,
 'bringing': 2338,
 'seamier': 40834,
 'wooded': 34703,
 'bravora': 52018,
 'grueling': 16817,
 'wooden': 1636,
 'wednesday': 16818,
 "'prix": 52019,
 'altagracia': 34704,
 'circuitry': 52020,
 'crotch': 11585,
 'busybody': 57766,
 "tart'n'tangy": 52021,
 'burgade': 14129,
 'thrace': 52023,
 "tom's": 11038,
 'snuggles': 52025,
 'francesco': 29114,
 'complainers': 52027,
 'templarios': 52125,
 '272': 40835,
 '273': 52028,
 'zaniacs': 52130,

This is the word list:

In [6]:
idx_arr = sorted(idx, key=idx.get)
# the first 10 words corresponding to idx 0 to 9
# in order of how frequently each word appeared in the given corpus
idx_arr[:10]

['the', 'and', 'a', 'of', 'to', 'is', 'br', 'in', 'it', 'i']

...and this is the mapping from id to word

In [7]:
# reverse index
idx2word = {v: k for k, v in idx.iteritems()}

We download the reviews using code copied from keras.datasets:

In [8]:
try:
    f = open('/home/xelburn/ML_Data/imdb/imdb_full.pkl')
except:
    path = get_file('imdb_full.pkl',
                origin='https://s3.amazonaws.com/text-datasets/imdb_full.pkl',
                md5_hash='d091312047c43cf9e4e38fef92437263')
    f = open(path, 'rb')
(x_train, labels_train), (x_test, labels_test) = pickle.load(f)

In [9]:
#the frist 5 reviews
print(x_train[:5])

[[23022, 309, 6, 3, 1069, 209, 9, 2175, 30, 1, 169, 55, 14, 46, 82, 5869, 41, 393, 110, 138, 14, 5359, 58, 4477, 150, 8, 1, 5032, 5948, 482, 69, 5, 261, 12, 23022, 73935, 2003, 6, 73, 2436, 5, 632, 71, 6, 5359, 1, 25279, 5, 2004, 10471, 1, 5941, 1534, 34, 67, 64, 205, 140, 65, 1232, 63526, 21145, 1, 49265, 4, 1, 223, 901, 29, 3024, 69, 4, 1, 5863, 10, 694, 2, 65, 1534, 51, 10, 216, 1, 387, 8, 60, 3, 1472, 3724, 802, 5, 3521, 177, 1, 393, 10, 1238, 14030, 30, 309, 3, 353, 344, 2989, 143, 130, 5, 7804, 28, 4, 126, 5359, 1472, 2375, 5, 23022, 309, 10, 532, 12, 108, 1470, 4, 58, 556, 101, 12, 23022, 309, 6, 227, 4187, 48, 3, 2237, 12, 9, 215], [23777, 39, 81226, 14, 739, 20387, 3428, 44, 74, 32, 1831, 15, 150, 18, 112, 3, 1344, 5, 336, 145, 20, 1, 887, 12, 68, 277, 1189, 403, 34, 119, 282, 36, 167, 5, 393, 154, 39, 2299, 15, 1, 548, 88, 81, 101, 4, 1, 3273, 14, 40, 3, 413, 1200, 134, 8208, 41, 180, 138, 14, 3086, 1, 322, 20, 4930, 28948, 359, 5, 3112, 2128, 1, 20045, 19339, 39, 8208, 45, 3

In [10]:
len(x_train)

25000

Here's the 1st review. As you see, the words have been replaced by ids. The ids can be looked up in idx2word.

In [11]:
# the first word is idx 23022
', '.join(map(str, x_train[0]))

'23022, 309, 6, 3, 1069, 209, 9, 2175, 30, 1, 169, 55, 14, 46, 82, 5869, 41, 393, 110, 138, 14, 5359, 58, 4477, 150, 8, 1, 5032, 5948, 482, 69, 5, 261, 12, 23022, 73935, 2003, 6, 73, 2436, 5, 632, 71, 6, 5359, 1, 25279, 5, 2004, 10471, 1, 5941, 1534, 34, 67, 64, 205, 140, 65, 1232, 63526, 21145, 1, 49265, 4, 1, 223, 901, 29, 3024, 69, 4, 1, 5863, 10, 694, 2, 65, 1534, 51, 10, 216, 1, 387, 8, 60, 3, 1472, 3724, 802, 5, 3521, 177, 1, 393, 10, 1238, 14030, 30, 309, 3, 353, 344, 2989, 143, 130, 5, 7804, 28, 4, 126, 5359, 1472, 2375, 5, 23022, 309, 10, 532, 12, 108, 1470, 4, 58, 556, 101, 12, 23022, 309, 6, 227, 4187, 48, 3, 2237, 12, 9, 215'

The first word of the first review is 23022. Let's see what that is.

In [12]:
# look up what the first word was: it is bromwell
idx2word[23022]

'bromwell'

Here's the whole review, mapped from ids to words.

In [13]:
# one by one convert all word idx's to their actual str values
# as a result of this you get tyhe original movie review in chars
' '.join([idx2word[o] for o in x_train[0]])

"bromwell high is a cartoon comedy it ran at the same time as some other programs about school life such as teachers my 35 years in the teaching profession lead me to believe that bromwell high's satire is much closer to reality than is teachers the scramble to survive financially the insightful students who can see right through their pathetic teachers' pomp the pettiness of the whole situation all remind me of the schools i knew and their students when i saw the episode in which a student repeatedly tried to burn down the school i immediately recalled at high a classic line inspector i'm here to sack one of your teachers student welcome to bromwell high i expect that many adults of my age think that bromwell high is far fetched what a pity that it isn't"

The labels are 1 for positive, 0 for negative.

In [14]:
# 1 for positive sentiment, 0 for negative sentiment
labels_train[:10]

[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]

Reduce vocab size by setting rare words to max index.

In [15]:
# get rid of relatively less frequent words / truncate to 5000
# so idx= 5000 will represent all less common words
vocab_size = 5000

trn = [np.array([i if i<vocab_size-1 else vocab_size-1 for i in s]) for s in x_train]
test = [np.array([i if i<vocab_size-1 else vocab_size-1 for i in s]) for s in x_test]

Look at distribution of lengths of sentences.

In [16]:
lens = np.array(map(len, trn))
# some reviews have 2493 words!, and on average a review has 237.7 words
(lens.max(), lens.min(), lens.mean())

(2493, 10, 237.71364)

Pad (with zero) or truncate each sentence to make every review of consistent length.

In [17]:
# truncate all reviews to 500 words
seq_len = 500

# keras function that pads sequences:
# it takes everything longer than max_len and truncates, and for those smaller,
# pads with value (below, we set this to 0)
trn = sequence.pad_sequences(trn, maxlen=seq_len, value=0)
test = sequence.pad_sequences(test, maxlen=seq_len, value=0)

This results in nice rectangular matrices that can be passed to ML algorithms. Reviews shorter than 500 words are pre-padded with zeros, those greater are truncated.

In [18]:
# now we have a nicely rectangular, 25k * 500 word matrix
# whcih we can pass to our machine learning model
trn.shape

(25000, 500)

In [19]:
# lets have a look at the first trn example, as you can see it has been padded
# (in the beginning with a lot of 0's
# keras by default pads to the left (beginning). Jeremy does not know whether
# padding to the right would make a diference
trn[0]

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,

## Create simple models

### Single hidden layer NN

The simplest model that tends to give reasonable results is a single hidden layer net. So let's try that. Note that we can't expect to get any useful results by feeding word ids directly into a neural net - so instead we use an embedding to replace them with a vector of 32 (initially random) floats for each word in the vocab.

#### Embedding

(For more info, look at Lesson4.ipynb)

Thefirst layer below is Embedding (and an Embedding Layer can only be the first layer), our inputs are word id's 
and we want to convert that to a vector and that is what an embedding does(rather than a 5000 column one-hot encoding)

So an Embedding says: 'look at that movie / user id and grab that vector directly'. 

It is just a computatioal in-memory shortcut to creating a one-hot encoding followed by a matrix product.

Below, the embedding returns a vector of length 32, so for each of the 500 words in each movie review you are getting a 32 element vector.

When Flatten() ed, you get 5000*32 weights (next layer)

#### Size of the Embedding

The choice of the size of the embedding (below it is 32), e.g. how many latent factors you want to associate with each word, is an architectural decision. It is problem-dependent, model-specific.

The size does not depend on the lenght of the word list but the complexity of the problem / language you are tryign to solve

Jeremy avoids the question, says: 'read the word2vec' paper and look at the 50d versus 200d embeddings and see how the accuracy changes.

In [24]:
model = Sequential([
    # !!! Every NLP model starts with an embedding layer  
    # Here each word, in the 5000, has been converted into a vector
    # of 32 latent factors    
    Embedding(vocab_size, 32, input_length=seq_len),
    Flatten(),
    Dense(100, activation='relu'),
    Dropout(0.7),
    Dense(1, activation='sigmoid')])
# We could have used softmax activation in the last layer, in which case, the labels
# we would have to convert to one-hot encoded versions: [0,1] and [1,0] and also
# use categorical cross entropy as the loss function

In [27]:
# if we were to use softmax aboe, we would have used categorical x-entropy here
# In looking at the nnet arcthitecture, 'None' always refers to minibatch size.
# For instance in the summary the size of the EMbedding layer is (None, 500, 32)
model.compile(loss='binary_crossentropy', optimizer=Adam(), metrics=['accuracy'])
model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
embedding_1 (Embedding)          (None, 500, 32)       160000      embedding_input_1[0][0]          
____________________________________________________________________________________________________
flatten_1 (Flatten)              (None, 16000)         0           embedding_1[0][0]                
____________________________________________________________________________________________________
dense_1 (Dense)                  (None, 100)           1600100     flatten_1[0][0]                  
____________________________________________________________________________________________________
dropout_1 (Dropout)              (None, 100)           0           dense_1[0][0]                    
___________________________________________________________________________________________

In [29]:
model.fit(trn, labels_train, validation_data=(test, labels_test), nb_epoch=2, batch_size=64)

Train on 25000 samples, validate on 25000 samples
Epoch 1/2
25000/25000 [==============================] - 0s - loss: 0.0852 - acc: 0.9730 - val_loss: 0.3578 - val_acc: 0.8665

The [stanford paper](http://ai.stanford.edu/~amaas/papers/wvSent_acl2011.pdf) that this dataset is from cites a state of the art accuracy (without unlabelled data) of 0.883. So we're short of that, but on the right track.

### Single conv layer with max pooling

A CNN is likely to work better, since it's designed to take advantage of ordered data. We'll need to use a 1D CNN, since a sequence of words is 1D.

In [15]:
# Using a 1D Convolution (since I am not looking at an image)!!!
# !!Instead of having 3 channels (RGB), I have 32 channels of embedding
# Initially used COnvolution size of 3 (as with 2d Conv, but realised)
# 5 worked better, so switched to 5, with 64 filters
conv1 = Sequential([
    Embedding(vocab_size, 32, input_length=seq_len, dropout=0.2), # doprout param zeroes at random 0.2 of the 32 embeddings
    Dropout(0.2), # removing at random some of the words (whole vector)
    Convolution1D(64, 5, border_mode='same', activation='relu'),
    Dropout(0.2),
    MaxPooling1D(),
    Flatten(),
    Dense(100, activation='relu'),
    Dropout(0.7),
    Dense(1, activation='sigmoid')])

In [16]:
conv1.compile(loss='binary_crossentropy', optimizer=Adam(), metrics=['accuracy'])

/home/xelburn/anaconda3/envs/py27/lib/python2.7/site-packages/theano/tensor/basic.py:2146: UserWarning: theano.tensor.round() changed its default from `half_away_from_zero` to `half_to_even` to have the same default as NumPy. Use the Theano flag `warn.round=False` to disable this warning.
  "theano.tensor.round() changed its default from"


In [17]:
conv1.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
embedding_1 (Embedding)          (None, 500, 32)       160000      embedding_input_1[0][0]          
____________________________________________________________________________________________________
dropout_1 (Dropout)              (None, 500, 32)       0           embedding_1[0][0]                
____________________________________________________________________________________________________
convolution1d_1 (Convolution1D)  (None, 500, 64)       10304       dropout_1[0][0]                  
____________________________________________________________________________________________________
dropout_2 (Dropout)              (None, 500, 64)       0           convolution1d_1[0][0]            
___________________________________________________________________________________________

In [18]:
conv1.fit(trn, labels_train, validation_data=(test, labels_test), nb_epoch=4, batch_size=64)

Train on 25000 samples, validate on 25000 samples
Epoch 1/4
25000/25000 [==============================] - 7s - loss: 0.5015 - acc: 0.7234 - val_loss: 0.2963 - val_acc: 0.8778

That's well past the Stanford paper's accuracy - another win for CNNs!

In [19]:
conv1.save_weights(model_path + 'conv1.h5')

In [20]:
conv1.load_weights(model_path + 'conv1.h5')

## Pre-trained vectors

### As of Dec 2016 CNN's with pretrain emdbeddings is the state of the art in NLP with deep nets.


### !!! Have a look at wordvectors.ipynb before moving on. That's where we preprocess the glove vectors.

In this section, we replicate the previous CNN, but using pre-trained embeddings.

Just like VGG, someone else has trained before, but words are much easier (the word dog is the word dog, no lighting or angles,) compared to images.

We can just load pretrained word vectors / embeddings download:

e.g.  http://nlp.stanford.edu/projects/glove/

or https://www.tensorflow.org/tutorials/word2vec/

These have been trained as unsupervised learning, tyring to capture the structure rather than fitting it to sentiment labels, etc, as we did above.

### !!! Jeremy: You should always use pretrained embeddings any time you do NLP. Never try to train something from scratch, it is a waste of time, and you wont have the same number of corpus (e.g. for the below GloVe embeddings, 6Billion words) and representation power.

This is the same idea as whenever looking at real world photos, start with VGG net if possible.

In [20]:
def load_vectors(loc):
    return (load_array(loc+'.dat'),
        pickle.load(open(loc+'_words.pkl','rb')),
        pickle.load(open(loc+'_idx.pkl','rb')))

In [21]:
vecs, words, wordidx = load_vectors('/home/xelburn/ML_Data/glove/results/6B.50d')

In [23]:
print(type(vecs))
print(vecs[0])
print(words[:5])

<type 'numpy.ndarray'>
[  4.1800e-01   2.4968e-01  -4.1242e-01   1.2170e-01   3.4527e-01  -4.4457e-02  -4.9688e-01
  -1.7862e-01  -6.6023e-04  -6.5660e-01   2.7843e-01  -1.4767e-01  -5.5677e-01   1.4658e-01
  -9.5095e-03   1.1658e-02   1.0204e-01  -1.2792e-01  -8.4430e-01  -1.2181e-01  -1.6801e-02
  -3.3279e-01  -1.5520e-01  -2.3131e-01  -1.9181e-01  -1.8823e+00  -7.6746e-01   9.9051e-02
  -4.2125e-01  -1.9526e-01   4.0071e+00  -1.8594e-01  -5.2287e-01  -3.1681e-01   5.9213e-04
   7.4449e-03   1.7778e-01  -1.5897e-01   1.2041e-02  -5.4223e-02  -2.9871e-01  -1.5749e-01
  -3.4758e-01  -4.5637e-02  -4.4251e-01   1.8785e-01   2.7849e-03  -1.8411e-01  -1.1514e-01
  -7.8581e-01]
['the', ',', '.', 'of', 'to']


The glove word ids and imdb word ids use different indexes. So we create a simple function that creates an embedding matrix using the indexes from imdb, and the embeddings from glove (where they exist).

In [24]:
def create_emb():
    ''' 
    map from one index to another index
    This is used to map the embeddings in imdb dataset to the embeddings coming
    from GloVe
    '''
    n_fact = vecs.shape[1]
    emb = np.zeros((vocab_size, n_fact))

    for i in range(1,len(emb)):
        word = idx2word[i]
        if word and re.match(r"^[a-zA-Z0-9\-]*$", word):
            src_idx = wordidx[word]
            emb[i] = vecs[src_idx]
        else:
            # If we can't find the word in glove, randomly initialize
            emb[i] = normal(scale=0.6, size=(n_fact,))

    # This is our "rare word" id - we want to randomly initialize
    emb[-1] = normal(scale=0.6, size=(n_fact,))
    emb/=3
    return emb

In [42]:
# Glove word vectors  indexed according to the imdb dataset
emb = create_emb()

In [43]:
print(type(emb))
print(len(emb))
print(len(emb[0]))
print(emb[:1])

<type 'numpy.ndarray'>
5000
50
[[ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
   0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
   0.  0.]]


### Pretrained vectors with CNN and Dropout

We pass our embedding matrix to the Embedding constructor, and set it to non-trainable.

In [40]:
print(emb.shape)
print(vocab_size)
print(seq_len)

Shape.0
5000
500


Initially when I tried the code below, I got this error: https://github.com/fchollet/keras/issues/4753. This error is a bug that happens because of "trainable=False". But apparently it only affects keras version 1.2.0 with THeano backend. It hsould work ok with Tensorlfow and keras version 2.0

In [56]:
# https://github.com/fchollet/keras/issues/4753
model = Sequential([
    Embedding(vocab_size, 50, input_length=seq_len, 
              weights=[emb], dropout=0.2),
    Dropout(0.25),
    Convolution1D(64, 5, border_mode='same', activation='relu'),
    Dropout(0.25),
    MaxPooling1D(),
    Flatten(),
    Dense(100, activation='relu'),
    Dropout(0.7),
    Dense(1, activation='sigmoid')])

In [57]:
model.compile(loss='binary_crossentropy', optimizer=Adam(), metrics=['accuracy'])

In [49]:
model.layers[0].trainable=False

In [58]:
model.fit(trn, labels_train, validation_data=(test, labels_test), nb_epoch=2, batch_size=64)

Train on 25000 samples, validate on 25000 samples
Epoch 1/2
25000/25000 [==============================] - 8s - loss: 0.4972 - acc: 0.7374 - val_loss: 0.3082 - val_acc: 0.8824

In [51]:
print( str(model.optimizer.lr.get_value() ))
model.optimizer.lr = 0.0001

0.0010000000475


In [52]:
model.fit(trn, labels_train, validation_data=(test, labels_test), 
          nb_epoch=5, batch_size=64)

Train on 25000 samples, validate on 25000 samples
Epoch 1/5
25000/25000 [==============================] - 8s - loss: 0.2707 - acc: 0.8907 - val_loss: 0.2484 - val_acc: 0.8998

In [62]:
print( str(model.optimizer.lr ))
# model.optimizerlr = 0.0001

0.0001


We already have beaten our previous model! But let's fine-tune the embedding weights - especially since the words we couldn't find in glove just have random embeddings.

In [53]:
model.layers[0].trainable=True
# no need to comile the model after making the first layer trainable
# since the architecture of the model hasnt changed

In [54]:
model.optimizer.lr=1e-4

In [55]:
model.fit(trn, labels_train, validation_data=(test, labels_test), nb_epoch=1, batch_size=64)

Train on 25000 samples, validate on 25000 samples
Epoch 1/1
25000/25000 [==============================] - 8s - loss: 0.2064 - acc: 0.9174 - val_loss: 0.2509 - val_acc: 0.8956

As expected, that's given us a nice little boost. :)

In [94]:
model.save_weights(model_path+'glove50.h5')

In [80]:
### Pretrained vectors with CNN and Dropout and BatchNorm

In [33]:
# I added the BatchNorm() layers below. After a COnvolution Layer, when you add 
# Batchnorm, you need to add the axis=1 parameter.
model = Sequential([
    Embedding(vocab_size, 50, input_length=seq_len, 
              weights=[emb], trainable=False),
    Dropout(0.25),
    BatchNormalization(axis=1),
    Convolution1D(64, 5, border_mode='same', activation='relu'),
    Dropout(0.25),
    MaxPooling1D(),
    BatchNormalization(axis=1),                
    Flatten(),
    Dense(100, activation='relu'),
    BatchNormalization(),        
    Dropout(0.7),
    Dense(1, activation='sigmoid')])

ValueError: You called `set_weights(weights)` on layer "embedding_5" with a  weight list of length 1, but the layer was expecting 0 weights. Provided weights: [array([[ 0.    ,  0.    ,  0.    , ...,  0.    , ...

## Multi-size CNN

This is an implementation of a multi-size CNN as shown in Ben Bowles' [excellent blog post](https://quid.com/feed/how-quid-uses-deep-learning-with-small-data).

They try not just one size, but multiple size convolutions.

In [23]:
from keras.layers import Merge

We use the functional API to create multiple conv layers of different sizes, and then concatenate them.

In [132]:
graph_in = Input ((vocab_size, 50))
convs = [ ] 
# Try usign COnvolutions of size 3 to 6
# for each, suing Functional API, add maxpooling and flatten
for fsz in range (3, 6): 
    x = Convolution1D(64, fsz, border_mode='same', activation="relu")(graph_in)
    x = MaxPooling1D()(x) 
    x = Flatten()(x) 
    convs.append(x)
# in the end, emrge all using concatenation
out = Merge(mode="concat")(convs) 
graph = Model(graph_in, out) 

In [174]:
emb = create_emb()

We then replace the conv/max-pool layer in our original CNN with the concatenated conv layers.

In [175]:
model = Sequential ([
    Embedding(vocab_size, 50, input_length=seq_len, dropout=0.2, weights=[emb]),
    Dropout (0.2),
    graph,
    Dropout (0.5),
    Dense (100, activation="relu"),
    Dropout (0.7),
    Dense (1, activation='sigmoid')
    ])

In [176]:
model.compile(loss='binary_crossentropy', optimizer=Adam(), metrics=['accuracy'])

In [177]:
model.fit(trn, labels_train, validation_data=(test, labels_test), nb_epoch=2, batch_size=64)

Train on 25000 samples, validate on 25000 samples
Epoch 1/2
25000/25000 [==============================] - 11s - loss: 0.3997 - acc: 0.8207 - val_loss: 0.3032 - val_acc: 0.8943
Epoch 2/2
25000/25000 [==============================] - 11s - loss: 0.2882 - acc: 0.8832 - val_loss: 0.2646 - val_acc: 0.9029


Interestingly, I found that in this case I got best results when I started the embedding layer as being trainable, and then set it to non-trainable after a couple of epochs. I have no idea why!

In [178]:
model.layers[0].trainable=False

In [179]:
model.optimizer.lr=1e-5

In [180]:
model.fit(trn, labels_train, validation_data=(test, labels_test), nb_epoch=2, batch_size=64)

Train on 25000 samples, validate on 25000 samples
Epoch 1/2
25000/25000 [==============================] - 11s - loss: 0.2556 - acc: 0.8949 - val_loss: 0.2534 - val_acc: 0.9024
Epoch 2/2
25000/25000 [==============================] - 11s - loss: 0.2360 - acc: 0.9057 - val_loss: 0.2577 - val_acc: 0.9036


This more complex architecture has given us another boost in accuracy.

## LSTM

We haven't covered this bit yet!
(Not using the trained Glove embeddings, no weights= emb in the first Embedding layer)

In [47]:
model = Sequential([
    Embedding(vocab_size, 32, input_length=seq_len, mask_zero=True,
              W_regularizer=l2(1e-6), dropout=0.2),
    # we want a single output so dont return sequences
    # each review is independent of each other so no statefulness.    
    LSTM(100, stateful=False, return_sequences=False,
         consume_less='gpu'),
    Dense(1, activation='sigmoid')])
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
embedding_6 (Embedding)          (None, 500, 32)       160000      embedding_input_6[0][0]          
____________________________________________________________________________________________________
lstm_1 (LSTM)                    (None, 100)           53200       embedding_6[0][0]                
____________________________________________________________________________________________________
dense_11 (Dense)                 (None, 1)             101         lstm_1[0][0]                     
Total params: 213,301
Trainable params: 213,301
Non-trainable params: 0
____________________________________________________________________________________________________


In [48]:
model.fit(trn, labels_train, validation_data=(test, labels_test), nb_epoch=5, batch_size=64)

Train on 25000 samples, validate on 25000 samples
Epoch 1/5
25000/25000 [==============================] - 64s - loss: 0.5705 - acc: 0.6934 - val_loss: 0.5819 - val_acc: 0.7801